# 📊 Line Plots & Two-way ANOVA
Analyze and visualize group differences with line plots and Two-way ANOVA in R.

---

**🗂️ Last updated:** 22 October 2025 

**🐳 Docker image:** `gnasello/datascience-env:2025-09-18`


### 📦 Load Required Packages


In [ ]:
library(ggplot2)
library(dataprepUtils)
library(ggplotUtils)
library(statsUtils)
library(rstatix)
library(lme4)
library(emmeans)

### 📁 Load and Prepare Data


In [ ]:
filetable <- "data.csv"

filename <- tools::file_path_sans_ext(filetable)

title <- 'Plot Title'
xlabel <- 'Time (day)'
ylabel <- 'Drug released (µg)'

# color of each group
scale_color_manual.values <- c("AS" = "#dca01cff", 
                               "AS+lapo" = "#386e28ff")

# Use the function to read and process the data
df <- read_and_process_data(filetable, 
                            x_col = "day", 
                            y_col = 'value',
                            group = 'condition',
                            #xlabels_ordered = names(scale_color_manual.values),
                           )
head(df)
tail(df)

## 🧹 Optional Data Manipulation *(commented out)*

### ✅ Option 1: Keep only specific values in a column

In [ ]:

# Uncomment and edit this section to keep only specific values in a chosen column
# values_to_keep <- c("Value1", "Value2")      # <-- Replace with values you want to keep
# column <- "ColumnName"                       # <-- Replace with the column name
# df <- subset(df, df[[column]] %in% values_to_keep)
# head(df)


### ❌ Option 2: Remove specific values from a column


In [ ]:

# Uncomment and edit this section to remove specific values from a chosen column
# values_to_remove <- c("Value1", "Value2")    # <-- Replace with values you want to remove
# column <- "ColumnName"                       # <-- Replace with the column name
# df <- subset(df, !(df[[column]] %in% values_to_remove))
# head(df)


### 📊 Line Plot


### 📈 Summarize Data by Group


The function below will be used to calculate the mean and the standard deviation, for the variable of interest, in each group. See [tutorial](http://www.sthda.com/english/wiki/ggplot2-line-plot-quick-start-guide-r-software-and-data-visualization#line-graph-with-error-bars)


In [ ]:
df_summary <- data_summary(df, varname="y", 
                           groupnames=c("group", "x"))
df_summary <- df_summary[complete.cases(df_summary$x),]

df_summary

### 🧪 Two-Way Mixed ANOVA


Read more on the theory of this method on [Datanovia](https://www.datanovia.com/en/lessons/mixed-anova-in-r/#two-way-mixed)


#### 🧮 ANOVA Computation


In [ ]:
# Two-way mixed ANOVA test
res.aov <- anova_test(
                      data = df, dv = y, wid = replicate,
                      between = group, within = x
                      )
get_anova_table(res.aov)

### 🔍 Post-hoc Analysis


#### 📌 Simple Main Effects


Explanation: Significant Two-Way Interaction


In [ ]:
# Effect of group at each time point
one.way <- df %>%
              group_by(x) %>%
              anova_test(dv = y, wid = replicate, between = group) %>%
              get_anova_table() %>%
              adjust_pvalue(method = "bonferroni")
one.way

Explanation: Non-Significant Two-Way Interaction


In [ ]:
df %>%
      pairwise_t_test(
        y ~ group, paired = TRUE, 
        p.adjust.method = "bonferroni"
      )

#### 🔬 Pairwise Comparisons


In [ ]:
# Pairwise comparisons between group levels
pwc <- df %>%
          group_by(x) %>%
          pairwise_t_test(y ~ group, p.adjust.method = "bonferroni")
pwc

### 📉 Plot with Error Bars


In [ ]:
color='#404040ff'

ylim <- c(0,12)
xlim <- c(0,16)
width=9
height=7.6

p <- ggplot() +
     geom_line(data=df_summary, aes(x=x, y=mean, color=group), linetype = "dashed", linewidth=1.5) +
     geom_point(data=df_summary, aes(x=x, y=mean, color=group), size=4.5, shape=15, stroke=2) +
     geom_errorbar(data=df_summary, aes(x=x, y=mean, ymin=mean-sd, ymax=mean+sd), width=.4, linewidth=0.75, color=color)

img <- apply_minimal_theme(p, width=width, height=height,
                           title=title,
                           plot.title = element_text(size = 24),
                           axis.text=element_text(size=24),
                           axis.title=element_text(size=26),
                           xlabel = xlabel, 
                           ylabel= ylabel, 
                           xlim = xlim,
                           ylim = ylim,
                           scale_color_manual.values = scale_color_manual.values,
                          ) + 
            theme(axis.line.x.bottom=element_line(linewidth=0.75),
                  axis.line.y.left=element_line(linewidth=0.75))#+
            # scale_x_continuous(breaks = seq(0, 16, by = 3))

img

## 📐 Add p-values to Plot

In [ ]:
pwc <- rstatix::add_y_position(pwc)

In [ ]:
# Use the maximum y.position from the preprocessed pwc data
max_annotation_y <- max(pwc$y.position, na.rm = TRUE)

y.buffer=0.1
min.y = 0
# Set y-limits with buffer
ylim_auto <- c(min.y, max_annotation_y + (y.buffer * max_annotation_y))

p_stats <- apply_minimal_theme(p  + ggpubr::stat_pvalue_manual(pwc, x = 'x', label.size = 8, label = "p.adj.signif") , width=width, height=height,
                           title=title,
                           plot.title = element_text(size = 24),
                           axis.text=element_text(size=24),
                           axis.title=element_text(size=26),
                           xlabel = xlabel, 
                           ylabel= ylabel, 
                           xlim = xlim,
                           ylim = ylim_auto,
                           scale_color_manual.values = scale_color_manual.values,
                          ) + 
            theme(axis.line.x.bottom=element_line(linewidth=0.75),
                  axis.line.y.left=element_line(linewidth=0.75))

p_stats

## 🧩 Arrange Plots Side-by-Side

In [ ]:

width_aligned <- 2 * width
options(repr.plot.width = width_aligned)

aligned_plots <- ggpubr::ggarrange(
  img, p_stats,
  nrow = 1,
  align = "hv",
  common.legend = FALSE
)



### 💾 Export Plots

In [ ]:

fileoutput <- paste(filename,'.png', sep='')
ggplot2::ggsave(file=fileoutput, plot=img, width=width, height=height, bg='white')

fileoutput <- paste(filename,'.svg', sep='')
ggplot2::ggsave(file=fileoutput, plot=img, width=width, height=height)


In [ ]:

fileoutput <- paste0(filename, "-barplot_stats.svg")
ggplot2::ggsave(file = fileoutput, plot = aligned_plots, width = width_aligned, height = height)

fileoutput <- paste0(filename, "-barplot_stats.png")
ggplot2::ggsave(file = fileoutput, plot = aligned_plots, width = width_aligned, height = height)
